# 🧠 Run-2 LIQUID NEURAL NETWORK 🚀

In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

In [42]:
# Define the Liquid Neural Network class
class LiquidNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, task='cls'):
        super(LiquidNeuralNetwork, self).__init__()
        self.task = task
        self.hidden_size = hidden_size

        # Define the liquid time constant (LTC) layer
        self.ltc = nn.RNN(input_size, hidden_size, batch_first=True)

        # Define the output layer
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Pass through the LTC layer
        _, h_n = self.ltc(x)

        # Use the last hidden state for prediction
        out = self.output_layer(h_n.squeeze(0))

        # Apply softmax for classification tasks
        if self.task == 'cls':
            out = torch.softmax(out, dim=1)
        return out

In [43]:
# Load and preprocess the Iris dataset for classification
def load_iris_data():
    iris = load_iris()
    X = iris.data
    y = iris.target

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Standardize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_val = torch.tensor(X_val, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    y_val = torch.tensor(y_val, dtype=torch.long)
    y_test = torch.tensor(y_test, dtype=torch.long)

    return X_train, X_val, X_test, y_train, y_val, y_test

# Load and preprocess the Boston Housing dataset for regression
def load_boston_data():
    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    y = raw_df.values[1::2, 2]

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Standardize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_val = torch.tensor(X_val, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
    y_val = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)
    y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

    return X_train, X_val, X_test, y_train, y_val, y_test

# Train the model
def train_model(model, X_train, y_train, X_val, y_val, task='cls', epochs=100, lr=0.01):
    criterion = nn.CrossEntropyLoss() if task == 'cls' else nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Create DataLoader
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    val_dataset = TensorDataset(X_val, y_val)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Training loop with checkpointing
    best_val_loss = np.inf
    for epoch in range(epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X.unsqueeze(1))  # Add sequence dimension for RNN
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                outputs = model(batch_X.unsqueeze(1))  # Add sequence dimension for RNN
                val_loss += criterion(outputs, batch_y).item()
        val_loss /= len(val_loader)

        # Save the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'best_model_{task}.pth')
            print(f"Saved new best model with validation loss: {val_loss:.4f}")

        print(f"Epoch [{epoch+1}/{epochs}], Validation Loss: {val_loss:.4f}")

# Evaluate the model
def evaluate_model(model, X_test, y_test, task='cls'):
    model.eval()  # Set the model to evaluation mode
    criterion = nn.CrossEntropyLoss() if task == 'cls' else nn.MSELoss()

    # Create DataLoader for testing
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    test_loss = 0.0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            outputs = model(batch_X.unsqueeze(1))  # Add sequence dimension for RNN
            test_loss += criterion(outputs, batch_y).item()
    test_loss /= len(test_loader)

    if task == 'cls':
        print(f'Test Accuracy: {1 - test_loss:.4f}')
    else:
        print(f'Test MSE: {test_loss:.4f}')
    return test_loss

In [50]:
# Classification task (Iris dataset)
print("Training and evaluating on Iris dataset (Classification)...")
X_train, X_val, X_test, y_train, y_val, y_test = load_iris_data()
model_cls = LiquidNeuralNetwork(input_size=4, hidden_size=10, output_size=3, task='cls')
train_model(model_cls, X_train, y_train, X_val, y_val, task='cls', epochs=200)
evaluate_model(model_cls, X_test, y_test, task='cls')

Training and evaluating on Iris dataset (Classification)...
Saved new best model with validation loss: 1.0613
Epoch [1/200], Validation Loss: 1.0613
Saved new best model with validation loss: 1.0217
Epoch [2/200], Validation Loss: 1.0217
Saved new best model with validation loss: 0.9833
Epoch [3/200], Validation Loss: 0.9833
Saved new best model with validation loss: 0.9465
Epoch [4/200], Validation Loss: 0.9465
Saved new best model with validation loss: 0.9142
Epoch [5/200], Validation Loss: 0.9142
Saved new best model with validation loss: 0.8881
Epoch [6/200], Validation Loss: 0.8881
Saved new best model with validation loss: 0.8688
Epoch [7/200], Validation Loss: 0.8688
Saved new best model with validation loss: 0.8525
Epoch [8/200], Validation Loss: 0.8525
Saved new best model with validation loss: 0.8394
Epoch [9/200], Validation Loss: 0.8394
Saved new best model with validation loss: 0.8228
Epoch [10/200], Validation Loss: 0.8228
Saved new best model with validation loss: 0.8049

0.6016202569007874

In [51]:
# Regression task (Boston Housing dataset)
print("\nTraining and evaluating on Boston Housing dataset (Regression)...")
X_train, X_val, X_test, y_train, y_val, y_test = load_boston_data()
model_reg = LiquidNeuralNetwork(input_size=13, hidden_size=10, output_size=1, task='reg')
train_model(model_reg, X_train, y_train, X_val, y_val, task='reg', epochs=200)
evaluate_model(model_reg, X_test, y_test, task='reg')


Training and evaluating on Boston Housing dataset (Regression)...
Saved new best model with validation loss: 544.9758
Epoch [1/200], Validation Loss: 544.9758
Saved new best model with validation loss: 503.3520
Epoch [2/200], Validation Loss: 503.3520
Saved new best model with validation loss: 452.6255
Epoch [3/200], Validation Loss: 452.6255
Saved new best model with validation loss: 393.3667
Epoch [4/200], Validation Loss: 393.3667
Saved new best model with validation loss: 333.2827
Epoch [5/200], Validation Loss: 333.2827
Saved new best model with validation loss: 279.2735
Epoch [6/200], Validation Loss: 279.2735
Saved new best model with validation loss: 235.1700
Epoch [7/200], Validation Loss: 235.1700
Saved new best model with validation loss: 200.2098
Epoch [8/200], Validation Loss: 200.2098
Saved new best model with validation loss: 171.0119
Epoch [9/200], Validation Loss: 171.0119
Saved new best model with validation loss: 147.3348
Epoch [10/200], Validation Loss: 147.3348
Sa

36.084742188453674

# 🧠 Run-1 LIQUID NEURAL NETWORK 🚀

## True Liquid Time-Constant (LTC) Cell

- Implemented proper continuous-time dynamics with learnable time constants
- Added decay factor based on tau parameter that controls information flow

---

## Enhanced Architecture

- Multi-layer capability for deeper networks
- Self-attention mechanism for capturing temporal relationships
- Skip connections and layer normalization for better gradient flow

---

## Robust Training Framework

- Learning rate scheduling with `ReduceLROnPlateau`
- Early stopping to prevent overfitting
- Gradient clipping to prevent exploding gradients
- AdamW optimizer with weight decay for regularization

---

## Comprehensive Evaluation

- Detailed metrics for both classification and regression
- Visualization of results (confusion matrices, prediction plots)
- Feature importance analysis for regression tasks

---

## Hyperparameter Tuning

- Simple grid search to find optimal model configuration
- Best model checkpointing

---

## Improved Data Handling

- Better error handling for dataset loading
- Proper input normalization
- Support for both sequence and non-sequence data formats

---

The code is now much more robust, handles edge cases better, and should provide significantly better performance on both classification and regression tasks.

In [ ]:
!pip install -q torchdiffeq

In [66]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, confusion_matrix
import datetime

class LiquidTimeConstantCell(nn.Module):
    """True Liquid Time-Constant (LTC) cell implementation"""
    def __init__(self, input_size, hidden_size, tau_init=1.0):
        super(LiquidTimeConstantCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Learnable time constant parameter (tau)
        self.tau = nn.Parameter(torch.ones(hidden_size) * tau_init)

        # Input projection
        self.input_proj = nn.Linear(input_size, hidden_size)

        # Hidden state projection (recurrent weights)
        self.hidden_proj = nn.Linear(hidden_size, hidden_size)

        # Output nonlinearity
        self.activation = nn.Tanh()

        # Regularization parameters
        self.dropout = nn.Dropout(0.2)

    def forward(self, x, hidden):
        # Input projection
        input_term = self.input_proj(x)

        # Recurrent projection
        hidden_term = self.hidden_proj(hidden)

        # Apply time constant dynamics: dh/dt = -h/τ + f(W_i·x + W_h·h)
        # This is a discretized version of the ODE
        decay = torch.exp(-1.0 / self.tau.abs())
        new_hidden = decay * hidden + (1 - decay) * self.activation(input_term + hidden_term)

        # Apply dropout for regularization
        new_hidden = self.dropout(new_hidden)

        return new_hidden

class ImprovedLiquidNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, task='cls', dropout=0.2):
        super(ImprovedLiquidNeuralNetwork, self).__init__()
        self.task = task
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Input normalization
        self.input_norm = nn.BatchNorm1d(input_size)

        # Multi-layer LTC cells
        self.ltc_cells = nn.ModuleList()
        self.ltc_cells.append(LiquidTimeConstantCell(input_size, hidden_size))
        for i in range(1, num_layers):
            self.ltc_cells.append(LiquidTimeConstantCell(hidden_size, hidden_size))

        # Attention mechanism for temporal dynamics
        self.attention = nn.MultiheadAttention(hidden_size, num_heads=4, dropout=dropout)

        # Output projection with skip connection
        self.pre_output = nn.Linear(hidden_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, output_size)

        # Dropout for regularization
        self.dropout = nn.Dropout(dropout)

        # Layer normalization
        self.layer_norm = nn.LayerNorm(hidden_size)

    def forward(self, x):
        # x shape: [batch_size, seq_len, input_size]
        batch_size, seq_len, _ = x.size()

        # Initialize hidden states
        hidden_states = [torch.zeros(batch_size, self.hidden_size, device=x.device) for _ in range(self.num_layers)]

        # Store outputs from all timesteps for attention
        seq_outputs = torch.zeros(seq_len, batch_size, self.hidden_size, device=x.device)

        # Process each timestep
        for t in range(seq_len):
            # Normalize input at each timestep
            x_t = x[:, t, :]
            if seq_len == 1:  # Handle single timestep case
                x_t = self.input_norm(x_t)

            # Process through LTC layers
            layer_input = x_t
            for i, ltc_cell in enumerate(self.ltc_cells):
                hidden_states[i] = ltc_cell(layer_input, hidden_states[i])
                layer_input = hidden_states[i]

            # Store the output from the last LTC layer
            seq_outputs[t] = hidden_states[-1]

        # Apply attention over the sequence outputs
        attn_output, _ = self.attention(seq_outputs, seq_outputs, seq_outputs)

        # Get the final output (either last timestep or attention-weighted)
        final_output = attn_output[-1]

        # Apply layer normalization and skip connection
        final_output = self.layer_norm(final_output + hidden_states[-1])

        # Final projection
        pre_output = F.relu(self.pre_output(final_output))
        pre_output = self.dropout(pre_output)
        out = self.output_layer(pre_output)

        # Apply appropriate activation for task
        if self.task == 'cls':
            out = F.log_softmax(out, dim=1)

        return out

    def init_hidden(self, batch_size, device):
        """Initialize hidden states"""
        return [torch.zeros(batch_size, self.hidden_size, device=device) for _ in range(self.num_layers)]

# Load and preprocess the Iris dataset for classification
def load_iris_data():
    from sklearn.datasets import load_iris

    iris = load_iris()
    X = iris.data
    y = iris.target

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Standardize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_val = torch.tensor(X_val, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    y_val = torch.tensor(y_val, dtype=torch.long)
    y_test = torch.tensor(y_test, dtype=torch.long)

    # Add sequence dimension if not already present
    if X_train.dim() == 2:
        X_train = X_train.unsqueeze(1)
        X_val = X_val.unsqueeze(1)
        X_test = X_test.unsqueeze(1)

    return X_train, X_val, X_test, y_train, y_val, y_test, iris.feature_names, iris.target_names

# Load and preprocess the Boston Housing dataset for regression
def load_boston_data():
    try:
        from sklearn.datasets import load_boston
        boston = load_boston()
        X = boston.data
        y = boston.target
        feature_names = boston.feature_names
    except:
        # Alternative if sklearn's boston dataset is deprecated
        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        try:
            raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
            X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
            y = raw_df.values[1::2, 2]
            feature_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE',
                            'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
        except:
            print("Boston dataset couldn't be loaded!")

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Standardize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_val = torch.tensor(X_val, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
    y_val = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)
    y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

    # Add sequence dimension if not already present
    if X_train.dim() == 2:
        X_train = X_train.unsqueeze(1)
        X_val = X_val.unsqueeze(1)
        X_test = X_test.unsqueeze(1)

    return X_train, X_val, X_test, y_train, y_val, y_test, feature_names

# Improved training function with learning rate scheduler and early stopping
def train_model(model, X_train, y_train, X_val, y_val, task='cls', epochs=300, lr=0.001,
                batch_size=32, patience=20, weight_decay=1e-5):
    if task == 'cls':
        criterion = nn.NLLLoss()  # Using NLLLoss with log_softmax
    else:
        criterion = nn.MSELoss()

    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5)

    # Create DataLoader
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    val_dataset = TensorDataset(X_val, y_val)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # For tracking training progress
    train_losses = []
    val_losses = []

    # Early stopping parameters
    best_val_loss = float('inf')
    best_epoch = 0
    early_stop_counter = 0

    # Training loop
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()

            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        # Learning rate scheduling
        scheduler.step(val_loss)
        # After scheduler.step(val_loss):
        if scheduler.is_better(val_loss, scheduler.best):
            print(f"Learning rate updated to: {optimizer.param_groups[0]['lr']:.6f}")

        # Print progress
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch
            early_stop_counter = 0
            # Save the best model
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
                'train_loss': train_loss
            }, f'best_model_{task}.pth')
            print(f"Saved new best model with validation loss: {val_loss:.4f}")
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs. Best epoch was {best_epoch+1} with val_loss {best_val_loss:.4f}")
            break

    # Load the best model
    checkpoint = torch.load(f'best_model_{task}.pth', weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])

    # Plot training curves
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'Training and Validation Loss Curves for {task.upper()} task')
    plt.legend()
    plt.savefig(f'loss_curves_{task}.png')
    plt.close()

    return model, train_losses, val_losses

# Enhanced evaluation function
def evaluate_model(model, X_test, y_test, task='cls', feature_names=None, target_names=None):
    model.eval()  # Set the model to evaluation mode

    # Create DataLoader for testing
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    all_preds = []
    all_targets = []

    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            outputs = model(batch_X)

            if task == 'cls':
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.numpy())
                all_targets.extend(batch_y.numpy())
            else:
                all_preds.extend(outputs.squeeze().numpy())
                all_targets.extend(batch_y.squeeze().numpy())

    # Prepare metrics
    if task == 'cls':
        # Classification metrics
        acc = accuracy_score(all_targets, all_preds)
        cm = confusion_matrix(all_targets, all_preds)

        print(f'Test Accuracy: {acc:.4f}')
        print("\nConfusion Matrix:")
        print(cm)

        # Plot confusion matrix
        plt.figure(figsize=(10, 8))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title('Confusion Matrix')
        plt.colorbar()

        if target_names is not None:
            classes = target_names
        else:
            classes = [f'Class {i}' for i in range(len(cm))]

        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)

        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.tight_layout()
        plt.savefig('confusion_matrix.png')
        plt.close()

        return acc
    else:
        # Regression metrics
        mse = mean_squared_error(all_targets, all_preds)
        r2 = r2_score(all_targets, all_preds)

        print(f'Test MSE: {mse:.4f}')
        print(f'Test R²: {r2:.4f}')

        # Feature importance analysis (for regression)
        if feature_names is not None:
            print("\nAttempting to analyze feature importance...")
            try:
                # Simple feature importance analysis based on input gradients
                importance = analyze_feature_importance(model, X_test, feature_names)

                # Plot feature importance
                plt.figure(figsize=(12, 6))
                plt.bar(range(len(importance)), importance)
                plt.xticks(range(len(importance)), feature_names, rotation=90)
                plt.title('Feature Importance')
                plt.tight_layout()
                plt.savefig('feature_importance.png')
                plt.close()
            except Exception as e:
                print(f"Could not analyze feature importance: {e}")

        # Plot predictions vs actual
        plt.figure(figsize=(10, 6))
        plt.scatter(all_targets, all_preds, alpha=0.5)
        plt.plot([min(all_targets), max(all_targets)], [min(all_targets), max(all_targets)], 'r--')
        plt.xlabel('Actual')
        plt.ylabel('Predicted')
        plt.title('Actual vs Predicted Values')
        plt.tight_layout()
        plt.savefig('regression_predictions.png')
        plt.close()

        return mse, r2

# Analyze feature importance through gradient-based methods
def analyze_feature_importance(model, X_test, feature_names):
    """Analyze feature importance by measuring input gradients"""
    importance = np.zeros(len(feature_names))
    X = X_test.clone().detach().requires_grad_(True)

    model.eval()
    output = model(X)
    output.sum().backward()

    # Compute importance as the mean absolute gradient
    importance = X.grad.abs().mean(dim=0).squeeze().numpy()

    return importance

In [ ]:
# CLASSIFICATION TASK - IRIS DATASET
print("\n" + "="*50)
print("TRAINING AND EVALUATING ON IRIS DATASET (CLASSIFICATION)")
print("="*50)

X_train, X_val, X_test, y_train, y_val, y_test, feature_names, target_names = load_iris_data()

# Set up tracking for hyperparameter tuning
best_val_acc = 0
best_hidden_size = 0
best_num_layers = 0

# Hyperparameter search
for hidden_size in [16, 32]:
    for num_layers in [1, 2]:
        print(f"\nTrying hidden_size={hidden_size}, num_layers={num_layers}")

        model_cls = ImprovedLiquidNeuralNetwork(
            input_size=4,
            hidden_size=hidden_size,
            output_size=3,
            num_layers=num_layers,
            task='cls',
            dropout=0.2
        )

        # Train the model
        model_cls, _, _ = train_model(
            model_cls,
            X_train,
            y_train,
            X_val,
            y_val,
            task='cls',
            epochs=200,
            lr=0.001,
            batch_size=16,
            patience=15
        )

        # Quick validation check
        model_cls.eval()
        with torch.no_grad():
            val_outputs = model_cls(X_val)
            _, val_preds = torch.max(val_outputs, 1)
            val_acc = accuracy_score(y_val.numpy(), val_preds.numpy())

        print(f"Validation accuracy: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_hidden_size = hidden_size
            best_num_layers = num_layers

# Train the final model with best hyperparameters
print(f"\nTraining final classification model with hidden_size={best_hidden_size}, num_layers={best_num_layers}")
final_model_cls = ImprovedLiquidNeuralNetwork(
    input_size=4,
    hidden_size=best_hidden_size,
    output_size=3,
    num_layers=best_num_layers,
    task='cls'
)

final_model_cls, _, _ = train_model(
    final_model_cls,
    X_train,
    y_train,
    X_val,
    y_val,
    task='cls',
    epochs=300
)

# Evaluate the final model
test_acc = evaluate_model(final_model_cls, X_test, y_test, task='cls', target_names=target_names)
print(f"Final Test Accuracy: {test_acc:.4f}")

# REGRESSION TASK - BOSTON HOUSING DATASET
print("\n" + "="*50)
print("TRAINING AND EVALUATING ON BOSTON HOUSING DATASET (REGRESSION)")
print("="*50)

try:
    X_train, X_val, X_test, y_train, y_val, y_test, feature_names = load_boston_data()

    # Set up tracking for hyperparameter tuning
    best_val_mse = float('inf')
    best_hidden_size = 0
    best_num_layers = 0

    # Hyperparameter search
    for hidden_size in [24, 32]:
        for num_layers in [1, 2]:
            print(f"\nTrying hidden_size={hidden_size}, num_layers={num_layers}")

            model_reg = ImprovedLiquidNeuralNetwork(
                input_size=13,
                hidden_size=hidden_size,
                output_size=1,
                num_layers=num_layers,
                task='reg',
                dropout=0.1
            )

            # Train the model
            model_reg, _, _ = train_model(
                model_reg,
                X_train,
                y_train,
                X_val,
                y_val,
                task='reg',
                epochs=200,
                lr=0.001,
                batch_size=16,
                patience=15
            )

            # Quick validation check
            model_reg.eval()
            with torch.no_grad():
                val_outputs = model_reg(X_val)
                val_mse = mean_squared_error(y_val.numpy(), val_outputs.numpy())

            print(f"Validation MSE: {val_mse:.4f}")

            if val_mse < best_val_mse:
                best_val_mse = val_mse
                best_hidden_size = hidden_size
                best_num_layers = num_layers

    # Train the final model with best hyperparameters
    print(f"\nTraining final regression model with hidden_size={best_hidden_size}, num_layers={best_num_layers}")
    final_model_reg = ImprovedLiquidNeuralNetwork(
        input_size=13,
        hidden_size=best_hidden_size,
        output_size=1,
        num_layers=best_num_layers,
        task='reg'
    )

    final_model_reg, _, _ = train_model(
        final_model_reg,
        X_train,
        y_train,
        X_val,
        y_val,
        task='reg',
        epochs=300
    )

    # Evaluate the final model
    test_mse, test_r2 = evaluate_model(
        final_model_reg,
        X_test,
        y_test,
        task='reg',
        feature_names=feature_names
    )
    print(f"Final Test MSE: {test_mse:.4f}, R²: {test_r2:.4f}")

except Exception as e:
    print(f"Error in regression task: {e}")
    print("Skipping Boston Housing dataset evaluation")

In [100]:
# CLASSIFICATION TASK - IRIS DATASET
print("\n" + "="*50)
print("TRAINING AND EVALUATING ON IRIS DATASET (CLASSIFICATION)")
print("="*50)

X_train, X_val, X_test, y_train, y_val, y_test, feature_names, target_names = load_iris_data()

# Set up tracking for hyperparameter tuning
best_val_acc = 0
best_hidden_size = 0
best_num_layers = 0
best_model_state = None

# Hyperparameter search
for hidden_size in [16, 32]:
    for num_layers in [1, 2]:
        print(f"\nTrying hidden_size={hidden_size}, num_layers={num_layers}")

        model_cls = ImprovedLiquidNeuralNetwork(
            input_size=4,
            hidden_size=hidden_size,
            output_size=3,
            num_layers=num_layers,
            task='cls',
            dropout=0.2
        )

        # Train the model
        model_cls, _, _ = train_model(
            model_cls,
            X_train,
            y_train,
            X_val,
            y_val,
            task='cls',
            epochs=200,
            lr=0.001,
            batch_size=16,
            patience=15
        )

        # Quick validation check
        model_cls.eval()
        with torch.no_grad():
            val_outputs = model_cls(X_val)
            _, val_preds = torch.max(val_outputs, 1)
            val_acc = accuracy_score(y_val.numpy(), val_preds.numpy())

        print(f"Validation accuracy: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_hidden_size = hidden_size
            best_num_layers = num_layers
            best_model_state = model_cls.state_dict()

            # Save the best model from hyperparameter search
            torch.save({
                'model_state_dict': best_model_state,
                'hyperparams': {
                    'hidden_size': best_hidden_size,
                    'num_layers': best_num_layers,
                    'task': 'cls',
                    'input_size': 4,
                    'output_size': 3
                },
                'val_accuracy': best_val_acc,
                'feature_names': feature_names,
                'target_names': target_names
            }, 'best_iris_model_from_search.pth')
            print(f"Saved new best model with validation accuracy: {best_val_acc:.4f}")

# Train the final model with best hyperparameters
print(f"\nTraining final classification model with hidden_size={best_hidden_size}, num_layers={best_num_layers}")
final_model_cls = ImprovedLiquidNeuralNetwork(
    input_size=4,
    hidden_size=best_hidden_size,
    output_size=3,
    num_layers=best_num_layers,
    task='cls'
)

# Option 1: Initialize with the best model from hyperparameter search
if best_model_state is not None:
    final_model_cls.load_state_dict(best_model_state)
    print("Initialized final model with best weights from hyperparameter search")

# Train the final model further
final_model_cls, train_losses, val_losses = train_model(
    final_model_cls,
    X_train,
    y_train,
    X_val,
    y_val,
    task='cls',
    epochs=300
)

# Save the final trained model with comprehensive metadata
torch.save({
    'model_state_dict': final_model_cls.state_dict(),
    'hyperparams': {
        'hidden_size': best_hidden_size,
        'num_layers': best_num_layers,
        'task': 'cls',
        'input_size': 4,
        'output_size': 3
    },
    'training_history': {
        'train_losses': train_losses,
        'val_losses': val_losses
    },
    'feature_names': feature_names,
    'target_names': target_names,
    'metadata': {
        'description': 'Improved Liquid Neural Network for Iris classification',
        'date_created': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
}, 'final_iris_model.pth')
print("Final model saved to 'final_iris_model.pth'")

# Evaluate the final model
test_acc = evaluate_model(final_model_cls, X_test, y_test, task='cls', target_names=target_names)
print(f"Final Test Accuracy: {test_acc:.4f}")


TRAINING AND EVALUATING ON IRIS DATASET (CLASSIFICATION)

Trying hidden_size=16, num_layers=1
Epoch [1/200], Train Loss: 1.1041, Val Loss: 1.0481
Saved new best model with validation loss: 1.0481
Epoch [2/200], Train Loss: 1.0574, Val Loss: 1.0114
Saved new best model with validation loss: 1.0114
Epoch [3/200], Train Loss: 1.0358, Val Loss: 0.9763
Saved new best model with validation loss: 0.9763
Epoch [4/200], Train Loss: 0.9955, Val Loss: 0.9430
Saved new best model with validation loss: 0.9430
Epoch [5/200], Train Loss: 0.9682, Val Loss: 0.9141
Saved new best model with validation loss: 0.9141
Epoch [6/200], Train Loss: 0.9133, Val Loss: 0.8861
Saved new best model with validation loss: 0.8861
Epoch [7/200], Train Loss: 0.9001, Val Loss: 0.8631
Saved new best model with validation loss: 0.8631
Epoch [8/200], Train Loss: 0.8705, Val Loss: 0.8359
Saved new best model with validation loss: 0.8359
Epoch [9/200], Train Loss: 0.8318, Val Loss: 0.8062
Saved new best model with validation 

In [172]:
# REGRESSION TASK - BOSTON HOUSING DATASET
print("\n" + "="*50)
print("TRAINING AND EVALUATING ON BOSTON HOUSING DATASET (REGRESSION)")
print("="*50)

try:
    X_train, X_val, X_test, y_train, y_val, y_test, feature_names = load_boston_data()

    # Set up tracking for hyperparameter tuning
    best_val_mse = float('inf')
    best_hidden_size = 0
    best_num_layers = 0
    best_model_state = None

    # Hyperparameter search
    for hidden_size in [24, 32]:
        for num_layers in [1, 2]:
            print(f"\nTrying hidden_size={hidden_size}, num_layers={num_layers}")

            model_reg = ImprovedLiquidNeuralNetwork(
                input_size=13,
                hidden_size=hidden_size,
                output_size=1,
                num_layers=num_layers,
                task='reg',
                dropout=0.1
            )

            # Train the model
            model_reg, _, _ = train_model(
                model_reg,
                X_train,
                y_train,
                X_val,
                y_val,
                task='reg',
                epochs=200,
                lr=0.001,
                batch_size=16,
                patience=15
            )

            # Quick validation check
            model_reg.eval()
            with torch.no_grad():
                val_outputs = model_reg(X_val)
                val_mse = mean_squared_error(y_val.numpy(), val_outputs.numpy())

            print(f"Validation MSE: {val_mse:.4f}")

            if val_mse < best_val_mse:
                best_val_mse = val_mse
                best_hidden_size = hidden_size
                best_num_layers = num_layers
                best_model_state = model_reg.state_dict()

                # Save the best model from hyperparameter search
                torch.save({
                    'model_state_dict': best_model_state,
                    'hyperparams': {
                        'hidden_size': best_hidden_size,
                        'num_layers': best_num_layers,
                        'task': 'reg',
                        'input_size': 13,
                        'output_size': 1
                    },
                    'val_mse': best_val_mse,
                    'feature_names': feature_names
                }, 'best_boston_model_from_search.pth')
                print(f"Saved new best model with validation MSE: {best_val_mse:.4f}")

    # Train the final model with best hyperparameters
    print(f"\nTraining final regression model with hidden_size={best_hidden_size}, num_layers={best_num_layers}")
    final_model_reg = ImprovedLiquidNeuralNetwork(
        input_size=13,
        hidden_size=best_hidden_size,
        output_size=1,
        num_layers=best_num_layers,
        task='reg'
    )

    # Option 1: Initialize with the best model from hyperparameter search
    if best_model_state is not None:
        final_model_reg.load_state_dict(best_model_state)
        print("Initialized final model with best weights from hyperparameter search")

    # Train the final model further
    final_model_reg, train_losses, val_losses = train_model(
        final_model_reg,
        X_train,
        y_train,
        X_val,
        y_val,
        task='reg',
        epochs=300
    )

    # Save the final trained model with comprehensive metadata
    torch.save({
        'model_state_dict': final_model_reg.state_dict(),
        'hyperparams': {
            'hidden_size': best_hidden_size,
            'num_layers': best_num_layers,
            'task': 'reg',
            'input_size': 13,
            'output_size': 1
        },
        'training_history': {
            'train_losses': train_losses,
            'val_losses': val_losses
        },
        'feature_names': feature_names,
        'metadata': {
            'description': 'Improved Liquid Neural Network for Boston housing price regression',
            'date_created': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
    }, 'final_boston_model.pth')
    print("Final model saved to 'final_boston_model.pth'")

    # Evaluate the final model
    test_mse, test_r2 = evaluate_model(
        final_model_reg,
        X_test,
        y_test,
        task='reg',
        feature_names=feature_names
    )
    print(f"Final Test MSE: {test_mse:.4f}, R²: {test_r2:.4f}")

except Exception as e:
    print(f"Error in regression task: {e}")
    print("Skipping Boston Housing dataset evaluation")


TRAINING AND EVALUATING ON BOSTON HOUSING DATASET (REGRESSION)

Trying hidden_size=24, num_layers=1
Epoch [1/200], Train Loss: 601.7182, Val Loss: 590.8112
Saved new best model with validation loss: 590.8112
Epoch [2/200], Train Loss: 612.4411, Val Loss: 541.4534
Saved new best model with validation loss: 541.4534
Epoch [3/200], Train Loss: 500.2608, Val Loss: 464.7749
Saved new best model with validation loss: 464.7749
Epoch [4/200], Train Loss: 434.9375, Val Loss: 380.8878
Saved new best model with validation loss: 380.8878
Epoch [5/200], Train Loss: 357.0971, Val Loss: 287.6704
Saved new best model with validation loss: 287.6704
Epoch [6/200], Train Loss: 253.6115, Val Loss: 194.1476
Saved new best model with validation loss: 194.1476
Epoch [7/200], Train Loss: 180.3155, Val Loss: 113.7186
Saved new best model with validation loss: 113.7186
Epoch [8/200], Train Loss: 112.1518, Val Loss: 66.7696
Saved new best model with validation loss: 66.7696
Epoch [9/200], Train Loss: 84.5962, V

In [173]:
# Function to load and use a saved model
def load_and_use_model(model_path):
    """
    Load a saved model and return it ready for inference

    Parameters:
    model_path (str): Path to the saved model file

    Returns:
    model (ImprovedLiquidNeuralNetwork): Loaded model ready for inference
    metadata (dict): Model metadata and information
    """
    # Load the checkpoint
    checkpoint = torch.load(model_path, weights_only=False)

    # Extract hyperparameters
    hyperparams = checkpoint['hyperparams']

    # Create a new model with the same architecture
    model = ImprovedLiquidNeuralNetwork(
        input_size=hyperparams['input_size'],
        hidden_size=hyperparams['hidden_size'],
        output_size=hyperparams['output_size'],
        num_layers=hyperparams['num_layers'],
        task=hyperparams['task']
    )

    # Load the state dictionary
    model.load_state_dict(checkpoint['model_state_dict'])

    # Set to evaluation mode
    model.eval()

    # Return the model and any metadata
    metadata = {k: v for k, v in checkpoint.items() if k != 'model_state_dict'}

    return model, metadata

# Example of how to use the loaded model for inference
def example_inference_cls(n_samples=5):
    print("\n" + "="*50)
    print("EXAMPLE: LOADING AND USING SAVED MODEL")
    print("="*50)

    try:
        # Load the iris model
        model, metadata = load_and_use_model('final_iris_model.pth')
        print("Loaded Iris classification model")
        print(f"Hyperparameters: {metadata['hyperparams']}")

        # Load a small sample of test data
        from sklearn.datasets import load_iris
        iris = load_iris()

        # Choose only 5 samples - several ways to do this:
        # Option 1: Take the first 5 samples
        # Set a random seed for reproducibility
        # np.random.seed(42)

        # Choose 5 random samples
        indices = np.random.choice(iris.data.shape[0], n_samples, replace=False)
        X_sample = iris.data[indices]
        y_sample = iris.target[indices]

        # Get the class names
        class_names = iris.target_names

        # Preprocess (same as during training)
        X_sample_tensor = torch.tensor(X_sample, dtype=torch.float32).unsqueeze(1)  # Add sequence dimension

        # Make predictions
        with torch.no_grad():
            outputs = model(X_sample_tensor)
            _, preds = torch.max(outputs, 1)

        # Initialize counts
        correct = 0
        total = len(y_sample)

        print("\nSample predictions:")

        for i in range(total):
            sample = X_sample[i]
            pred_class = metadata['target_names'][preds[i]]  # Predicted class name
            actual_class = class_names[y_sample[i]]  # Actual class name

            print(f"Sample {i+1}: Features={sample.round(2)}, Predicted class={pred_class}, Actual class={actual_class}")

            # Compare prediction with actual class
            if preds[i] == y_sample[i]:
                correct += 1

        # Calculate accuracy
        accuracy = correct / total * 100
        print(f"\nClassification Accuracy: {accuracy:.2f}%")
    except Exception as e:
        print(f"Error in example inference: {e}")

In [140]:
example_inference_cls(n_samples=10)


EXAMPLE: LOADING AND USING SAVED MODEL
Loaded Iris classification model
Hyperparameters: {'hidden_size': 32, 'num_layers': 2, 'task': 'cls', 'input_size': 4, 'output_size': 3}

Sample predictions:
Sample 1: Features=[5.  2.3 3.3 1. ], Predicted class=virginica, Actual class=versicolor
Sample 2: Features=[5.2 3.5 1.5 0.2], Predicted class=setosa, Actual class=setosa
Sample 3: Features=[6.7 3.3 5.7 2.1], Predicted class=virginica, Actual class=virginica
Sample 4: Features=[6.2 2.8 4.8 1.8], Predicted class=virginica, Actual class=virginica
Sample 5: Features=[6.9 3.1 4.9 1.5], Predicted class=virginica, Actual class=versicolor
Sample 6: Features=[5.1 3.7 1.5 0.4], Predicted class=setosa, Actual class=setosa
Sample 7: Features=[4.9 3.6 1.4 0.1], Predicted class=setosa, Actual class=setosa
Sample 8: Features=[7.7 3.8 6.7 2.2], Predicted class=virginica, Actual class=virginica
Sample 9: Features=[6.  2.2 5.  1.5], Predicted class=virginica, Actual class=virginica
Sample 10: Features=[5.4 3

In [174]:
# Example inference function
def example_inference_reg(feature_names, n_samples=5):
    print("\n" + "="*50)
    print("EXAMPLE: LOADING AND USING SAVED REGRESSION MODEL")
    print("="*50)

    try:
        # Load the California housing model
        model, metadata = load_and_use_model('final_boston_model.pth')
        print("Loaded California Housing regression model")
        print(f"Hyperparameters: {metadata['hyperparams']}")

        # Create some sample data
        from sklearn.datasets import fetch_california_housing
        from sklearn.preprocessing import StandardScaler

        # Alternative if sklearn's boston dataset is deprecated
        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        y = raw_df.values[1::2, 2]

        # Choose only 5 samples - several ways to do this:
        # Option 1: Take the first 5 samples
        # Set a random seed for reproducibility
        # np.random.seed(42)

        # Choose 5 random samples
        indices = np.random.choice(len(X), n_samples, replace=False)
        X_sample = X[indices]
        y_actual = y[indices]

        # Preprocess (same as during training)
        scaler = StandardScaler()
        scaler.fit(X_sample)  # Fit on all data since we don't have the exact scaler from training
        X_sample_scaled = scaler.transform(X_sample)
        X_sample_tensor = torch.tensor(X_sample_scaled, dtype=torch.float32).unsqueeze(1)  # Add sequence dimension

        # Make predictions
        with torch.no_grad():
            predictions = model(X_sample_tensor)
            predictions = predictions.numpy().flatten()

        # Print results
        print("\nSample predictions:")
        print(f"{'Feature Values':<60} | {'Actual':<10} | {'Predicted':<10}")
        print("-" * 85)

        for i in range(len(X_sample)):
            features_str = ", ".join([f"{name}={val:.2f}" for name, val in zip(feature_names, X_sample[i])])
            print(f"{features_str:<60} | {y_actual[i]:<10.2f} | {predictions[i]:<10.2f}")

        # Calculate error metrics for these samples
        sample_mse = mean_squared_error(y_actual, predictions)
        sample_r2 = r2_score(y_actual, predictions)
        print(f"\nSample MSE: {sample_mse:.4f}, Sample R²: {sample_r2:.4f}")

    except Exception as e:
        print(f"Error in example inference: {e}")

In [177]:
feature_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE',
                        'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
example_inference_reg(feature_names, n_samples=20)


EXAMPLE: LOADING AND USING SAVED REGRESSION MODEL
Loaded California Housing regression model
Hyperparameters: {'hidden_size': 32, 'num_layers': 1, 'task': 'reg', 'input_size': 13, 'output_size': 1}

Sample predictions:
Feature Values                                               | Actual     | Predicted 
-------------------------------------------------------------------------------------
CRIM=8.79, ZN=0.00, INDUS=18.10, CHAS=0.00, NOX=0.58, RM=5.57, AGE=70.60, DIS=2.06, RAD=24.00, TAX=666.00, PTRATIO=20.20, B=3.65, LSTAT=17.16 | 11.70      | 13.79     
CRIM=0.09, ZN=0.00, INDUS=12.83, CHAS=0.00, NOX=0.44, RM=6.14, AGE=45.80, DIS=4.09, RAD=5.00, TAX=398.00, PTRATIO=18.70, B=386.96, LSTAT=10.27 | 20.80      | 21.28     
CRIM=0.03, ZN=0.00, INDUS=5.19, CHAS=0.00, NOX=0.52, RM=5.87, AGE=46.30, DIS=5.23, RAD=5.00, TAX=224.00, PTRATIO=20.20, B=396.90, LSTAT=9.80 | 19.50      | 19.33     
CRIM=9.82, ZN=0.00, INDUS=18.10, CHAS=0.00, NOX=0.67, RM=6.79, AGE=98.80, DIS=1.36, RAD=24.00, TAX=666.

In [178]:
# Enhanced evaluation function
def eval_liquid_regression(model_path, X_test, y_test, task='cls', feature_names=None, target_names=None):
    # Load the California housing model
    model, metadata = load_and_use_model(model_path)
    print(f"Hyperparameters: {metadata['hyperparams']}")
    model.eval()  # Set the model to evaluation mode

    # Create DataLoader for testing
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    all_preds = []
    all_targets = []

    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            outputs = model(batch_X)

            if task == 'cls':
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.numpy())
                all_targets.extend(batch_y.numpy())
            else:
                all_preds.extend(outputs.squeeze().numpy())
                all_targets.extend(batch_y.squeeze().numpy())

    # Prepare metrics
    if task == 'cls':
        # Classification metrics
        acc = accuracy_score(all_targets, all_preds)
        cm = confusion_matrix(all_targets, all_preds)

        print(f'Test Accuracy: {acc:.4f}')
        print("\nConfusion Matrix:")
        print(cm)

        return acc
    else:
        # Regression metrics
        mse = mean_squared_error(all_targets, all_preds)
        r2 = r2_score(all_targets, all_preds)

        print(f'Test MSE: {mse:.4f}')
        print(f'Test R²: {r2:.4f}')

        return mse, r2

In [179]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

# Train and evaluate Linear Regression
def linear_regression(X_train, X_test, y_train, y_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    #print(f'Linear Regression MSE: {mse:.4f}')
    #print(f'Linear Regression R2: {r2:.4f}')
    return mse, r2

# Train and evaluate Polynomial Regression
def polynomial_regression(X_train, X_test, y_train, y_test, degree=2):
    poly = PolynomialFeatures(degree=degree)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_poly, y_train)
    y_pred = model.predict(X_test_poly)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    #print(f'Polynomial Regression (degree={degree}) MSE: {mse:.4f}')
    #print(f'Polynomial Regression (degree={degree}) R2: {r2:.4f}')
    return mse, r2

# Train and evaluate Support Vector Regression (SVR)
def support_vector_regression(X_train, X_test, y_train, y_test):
    model = SVR(kernel='rbf')
    model.fit(X_train, y_train.ravel())  # Reshape y_train to 1D
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    #print(f'SVR MSE: {mse:.4f}')
    #print(f'SVR R2: {r2:.4f}')
    return mse, r2

In [181]:
X_test.shape

torch.Size([102, 1, 13])

In [182]:
# Test and evaluate Liquid Neural Network (Reg.)
import torch

model_reg = "final_boston_model.pth"

if not isinstance(X_test, torch.Tensor):
  X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)

if not isinstance(y_test, torch.Tensor):
    y_test = torch.tensor(y_test, dtype=torch.float32)

lnn_mse, lnn_r2 = eval_liquid_regression(model_reg, X_test, y_test, task='reg')

Hyperparameters: {'hidden_size': 32, 'num_layers': 1, 'task': 'reg', 'input_size': 13, 'output_size': 1}
Test MSE: 14.4078
Test R²: 0.8035


In [183]:
# Tensor to Numpy
import torch

if isinstance(X_train, torch.Tensor):
    print("X_train is a PyTorch tensor.")
    X_train=X_train.squeeze(1).cpu().detach().numpy()
    y_train=y_train.squeeze(1).cpu().detach().numpy()

    X_test=X_test.squeeze(1).cpu().detach().numpy()
    y_test=y_test.squeeze(1).cpu().detach().numpy()
else:
    print("X_train is NOT a PyTorch tensor.")

X_train is a PyTorch tensor.


In [184]:
# Train and evaluate Linear Regression
lr_mse, lr_r2 = linear_regression(X_train, X_test, y_train, y_test)

# Train and evaluate Polynomial Regression
poly_mse, poly_r2 = polynomial_regression(X_train, X_test, y_train, y_test, degree=2)

# Train and evaluate SVR
svr_mse, svr_r2 = support_vector_regression(X_train, X_test, y_train, y_test)

# Compare performance
print("\nPerformance Comparison:")
print(f"Liquid Neural Network (Reg.) MSE: {lnn_mse:.4f} || R2: {lnn_r2:.4f}")
print(f"Linear Regression MSE: {lr_mse:.4f} || R2: {lr_r2:.4f}")
print(f"Polynomial Regression MSE: {poly_mse:.4f} || R2: {poly_r2:.4f}")
print(f"SVR MSE: {svr_mse:.4f} || R2: {svr_r2:.4f}")


Performance Comparison:
Liquid Neural Network (Reg.) MSE: 14.4078 || R2: 0.8035
Linear Regression MSE: 25.1021 || R2: 0.6577
Polynomial Regression MSE: 13.6509 || R2: 0.8139
SVR MSE: 29.0280 || R2: 0.6042
